In [1]:
import torch, transformers
import re

class MultiTokenMaskLM:
    
    def __init__(self, model_name="roberta-large"):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(model_name)


    def get_log_probability(self, text, text_to_mask):
        orig_tokens, masked_tokens = self._tokenize_and_mask(text, text_to_mask)
        
        mask_ids = [i for i in range(len(orig_tokens["input_ids"][0])) 
                    if masked_tokens["input_ids"][0,i]==self.tokenizer.mask_token_id]    
        
        total_log_prob = 0
        with torch.no_grad():
            for mask_id in mask_ids[::-1]:
                logits = self.model(**masked_tokens).logits
                log_probs = torch.nn.functional.log_softmax(logits[0,mask_id], dim=0)
                actual_id = orig_tokens["input_ids"][0,mask_id]
                total_log_prob += log_probs[actual_id].item()
                masked_tokens["input_ids"][0, mask_id] = actual_id
                
        return total_log_prob
        
            
    def get_alternatives(self, text, text_to_mask, beam_size=20):

        _, tokens = self._tokenize_and_mask(text, text_to_mask)

        mask_ids = [i for i in range(len(tokens["input_ids"][0])) 
                    if tokens["input_ids"][0,i]==self.tokenizer.mask_token_id]
            
        beam = [(tokens,0)]
        
        for mask_id in mask_ids[::-1]:
            
            new_beam = []  
            for current, current_logprob in beam:           
                for filled, new_logprob in self._fill(current, mask_id, beam_size):
                    new_beam.append((filled, current_logprob + new_logprob))
            
            beam = sorted(new_beam, key=lambda x : x[1])[-beam_size:]
            print("best replacement so far:", self.tokenizer.decode(beam[-1][0]["input_ids"][0,mask_ids]))
        
        beam_string = {self.tokenizer.decode(solution["input_ids"][0, mask_ids]):logprob 
                       for solution, logprob in beam[::-1]}
        
        return beam_string      


    def _tokenize_and_mask(self, text, text_to_mask):
        if text_to_mask is not None and text_to_mask not in text:
            raise RuntimeError("Text to mask must be included in full text")

        orig_tokens = self.tokenizer(text, return_offsets_mapping=True, return_tensors="pt")
        offset_mapping = orig_tokens["offset_mapping"][0]
        del orig_tokens["offset_mapping"]
    #    print("tokens:", [self.tokenizer.decode(x) for x in orig_tokens["input_ids"][0]])
        
        masked_tokens = {key:value.clone().detach() for key, value in orig_tokens.items()}
        for match in re.finditer(re.escape(text_to_mask), text):
            for i, (tok_start, tok_end) in enumerate(offset_mapping):
                if tok_start >= match.start(0) and tok_end <= match.end(0) and tok_end > tok_start:
                    masked_tokens["input_ids"][0,i] = self.tokenizer.mask_token_id
        
        return orig_tokens, masked_tokens
           
           
    def _fill(self, tokens, mask_id, beam_size=100):
        
        with torch.no_grad():
            logits = self.model(**tokens).logits
            log_probs = torch.nn.functional.log_softmax(logits[0,mask_id], dim=0)

            best_replacements_idx = torch.argsort(log_probs)[-beam_size:]    
            
            for replacement_id in best_replacements_idx:
                new_tokens = {key:value.clone().detach() for key, value in tokens.items()}
                new_tokens["input_ids"][0, mask_id] = replacement_id
                logprob = log_probs[replacement_id].item()
                yield new_tokens, logprob

        
filler = MultiTokenMaskLM(model_name="roberta-large")

text = """Charles Darwin (12 February 1809 – 19 April 1882) was an English naturalist, geologist, and biologist, widely known for contributing to the understanding of evolutionary biology."""
print("Log probability:", filler.get_log_probability(text, "Charles Darwin"))
print("Alternative results:", filler.get_alternatives(text, "Charles Darwin"))


/home/plison/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Log probability: -2.7288677991600707


2022-11-25 20:15:45.350769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 20:15:46.931706: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


best replacement so far: <mask> Darwin
